In [11]:
# import libraries

import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib import request
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import time
import re

In [3]:
# Go to URL for inmate/paroleee locator
main_page_url = 'http://inmatelocator.cor.pa.gov/#/'
driver = webdriver.Safari()
driver.get(main_page_url)

In [4]:
def query_selector(selector,action='.click()'):
    """
    convenience function for clicking buttons
    """
    
    driver.execute_script(f'document.querySelector("{selector}"){action}')

In [5]:
# remove the popup on initial page load
query_selector("#app > app-footer > app-disclamiermodal > div > div > div > div > div.app-modal-footer > button")

In [6]:
# scroll to visible fields
driver.execute_script("window.scrollTo(0,200)")

In [7]:
# css-selector strings for dropdowns
committing_counties_str = "#committingCounty > option:nth-child({})"
locations_str = "#currentLocation > option:nth-child({})"

# css-selector strings for submit and back buttons
submit_button = "#form > div.search-results > div > div:nth-child(2) > button"
back_button = "#app > app-searchresults > div > button"

In [8]:
# get the dropdown option indices for all SCI (so don't have to run driver.find_elements_by_id on every loop)
SCI_list = ['ALBION','BENNER TOWNSHIP','CAMBRIDGE SPRINGS','CAMP HILL','CHESTER','COAL TOWNSHIP',
            'DALLAS','FAYETTE','FOREST','FRACKVILLE','GREENE','HOUTZDALE','HUNTINGDON','LAUREL HIGHLANDS',
            'MAHANOY','MERCER','MUNCY','PHOENIX','PINE GROVE','QUEHANNA BOOT CAMP','ROCKVIEW','SMITHFIELD','SOMERSET','WAYMART']

SCI_location_indices = []
for i,tag in enumerate(driver.find_element_by_id("currentLocation").find_elements_by_tag_name('option')):
    if tag.text in SCI_list:
        SCI_location_indices.append(i+1)

In [12]:
rows = []

# for every county
for c in tqdm(range(2,69)):
    # for every SCI
    for l in SCI_location_indices:
        # 1.find county and SCI dropdown elements
        county = driver.find_element_by_css_selector(committing_counties_str.format(c))
        location = driver.find_element_by_css_selector(locations_str.format(l))
        
        # grab text
        ctext = county.text
        ltext = location.text
        
        # select dropdowns
        county.click()
        location.click()
        
        # submit form
        query_selector(submit_button)
        time.sleep(1)
        
        # wait for screen to load
        hold = True
        while hold:
            try:
                # 2. once loaded grab the total-results element at bottom of screen
                text = driver.find_elements_by_class_name('total-results')[1].text
                time.sleep(0.1)
                hold = False
            except:
                continue 
        
        # 3. go back to main page
        query_selector(back_button)
        time.sleep(0.5)
        
        # wait for screen to load
        hold = True
        while hold:
            try:
                # 4. scroll to visible dropdowns
                driver.execute_script("window.scrollTo(0,200)")
                time.sleep(0.5)
                hold = False
            except:
                continue
        
        # 5. save information
        rows.append({
            "county":ctext,
            "location":ltext,
            "count_text":text
        })


100%|██████████| 67/67 [1:02:41<00:00, 56.14s/it]


In [14]:
DF = pd.DataFrame(rows)


In [24]:
def parse_count(st):
    count = st.split()[-2]
    if count == 'results':
        return 0
    else:
        return int(count)
    
DF['incarcerated_persons'] = DF.count_text.apply(parse_count)

In [26]:
DF.to_csv('../../data/comparing_SCI-County/County_pop_over_SCI.csv')